<a href="https://colab.research.google.com/github/zelal-Eizaldeen/project2_Walmart_Price_Prediction/blob/main/Assignmnet_5_zelalae2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix


# **Team Members**

Zilal Eiz Aldin (Online MCS) Zelalae2@illinois.edu

Mesay Taye (Online MCS) mesayst2@illinois.edu

Jianci Zhai (Online MCS) jianciz2@illinois.edu

# **Contribution of each member**
Final Check By Mesay

Preliminary Function By Zilal

Debugging / Prediction / Testing By Jianci

https://campuswire.com/c/GB46E5679/feed/834

https://liangfgithub.github.io/Coding/F24_Coding5.pdf

##**Part 1 -- By Zilal**
- Design SVM From Scratch


In [2]:
DATA_DIR='/content/drive/MyDrive/MastersDegree/CS598 PSL/Assignments/A5/Data'

In [6]:
import numpy as np

class LinearSVM:
    def __init__(self, lambda_param, epochs):
       # self.learning_rate = learning_rate
        self.lambda_param = lambda_param
        self.epochs = epochs

    def fit(self, X, y):
        # Initialize parameters
        self.w =  np.zeros(X.shape[1])  # Random initialization of weights
        self.b = 0  # Bias initialized to 0
        self.t = 0  # Iteration counter

        # Stochastic Gradient Descent
        for epoch in range(self.epochs):
            for i in range(X.shape[0]):
                xi = X.loc[i, :]
                yi = y[i]
                self.t += 1

                # Compute the hinge loss gradient
                if yi * (np.dot(xi, self.w) + self.b) < 1:
                    dw = self.w - yi * xi
                    db = -yi
                else:
                    dw = self.w
                    db = 0

                # Add regularization term (derivative of lambda * ||w||^2)
                dw += 2 * self.lambda_param * self.w

                # Update parameters
                learning_rate  = 1 / (self.lambda_param*self.t)
                self.w -= learning_rate * dw
                self.b -= learning_rate * db

            # Optionally, print the loss after each epoch to track progress
            loss = self.compute_loss(X, y)
            print(f"Epoch {epoch+1}/{self.epochs}, Loss: {loss}")

    def compute_loss(self, X, y):
        # Compute the hinge loss + regularization
        margins = 1 - y * (np.dot(X, self.w) + self.b)
        hinge_loss = np.sum(np.maximum(0, margins)) / X.shape[0]
        reg_loss = self.lambda_param * np.sum(self.w ** 2)
        return hinge_loss + reg_loss

    def predict(self, X):
        # Predict the class labels: 1 if positive, -1 if negative
        return np.sign(np.dot(X, self.w) + self.b)


# **Jianci - Testing SVM**

In [33]:
##Testing
# Example usage:
train_org = pd.read_csv('https://liangfgithub.github.io/Data/coding5_train.csv')
test =  pd.read_csv('https://liangfgithub.github.io/Data/coding5_test.csv')




In [58]:
lambda_param_range = np.linspace(0.001, 4, 500)

In [59]:
  train =train_org.sample(frac=1, random_state=42).reset_index(drop=True)


  Y=(train['Y']>5).astype(int)*2-1
  X_train=train.drop(['Y'] , axis=1)


  Y_test=(test['Y']>5).astype(int)*2-1
  X_test=test.drop(['Y'] , axis=1)

  lambda_param = lambda_param_range[1]

  print(f"lambda ={lambda_param}")
  model = LinearSVM(lambda_param= lambda_param_range[1], epochs=50)
  model.fit(X_train, Y)
  predictions = model.predict(X_test)
  error_rate = 1-(predictions == Y_test).sum()/len(Y_test)
  cm = confusion_matrix(Y_test, predictions)
  print(f"Error Rate: {error_rate}")
  print(f"Confusion Matrix:\n{cm}")

lambda =0.009014028056112226
Epoch 1/50, Loss: 5.196689693902752
Epoch 2/50, Loss: 4.423204957598478
Epoch 3/50, Loss: 3.030449806892454
Epoch 4/50, Loss: 0.7773570366488896
Epoch 5/50, Loss: 2.631316655151184
Epoch 6/50, Loss: 1.454798154225153
Epoch 7/50, Loss: 0.3497092327917698
Epoch 8/50, Loss: 0.31091409665466496
Epoch 9/50, Loss: 0.3702398051463517
Epoch 10/50, Loss: 0.13804596487397458
Epoch 11/50, Loss: 0.7117045289723678
Epoch 12/50, Loss: 0.2230126645652708
Epoch 13/50, Loss: 0.1538206398181963
Epoch 14/50, Loss: 0.25676850111610616
Epoch 15/50, Loss: 0.3058409342623671
Epoch 16/50, Loss: 0.09623521884507338
Epoch 17/50, Loss: 0.11964380570271782
Epoch 18/50, Loss: 0.12962132713668967
Epoch 19/50, Loss: 0.09181988206879893
Epoch 20/50, Loss: 0.06708817154993943
Epoch 21/50, Loss: 0.11321975916909675
Epoch 22/50, Loss: 0.06100561047664533
Epoch 23/50, Loss: 0.2596544047129705
Epoch 24/50, Loss: 0.07964928875234337
Epoch 25/50, Loss: 0.053349958057578445
Epoch 26/50, Loss: 0.0

In [60]:
epochs =50

for lambda_param in lambda_param_range:

  #random shuffle the training data
  train =train_org.sample(frac=1, random_state=42).reset_index(drop=True)


  Y=(train['Y']>5).astype(int)*2-1
  X_train=train.drop(['Y'] , axis=1)


  Y_test=(test['Y']>5).astype(int)*2-1
  X_test=test.drop(['Y'] , axis=1)

  print(f"lambda ={lambda_param}")

  model = LinearSVM(lambda_param=lambda_param, epochs=epochs)
  model.fit(X_train, Y)
  predictions = model.predict(X_test)
  error_rate = 1-(predictions == Y_test).sum()/len(Y_test)
  cm = confusion_matrix(Y_test, predictions)
  print(f"Error Rate: {error_rate}")
  print(f"Confusion Matrix:\n{cm}")


lambda =0.001
Epoch 1/50, Loss: inf
Epoch 2/50, Loss: inf
Epoch 3/50, Loss: inf
Epoch 4/50, Loss: inf
Epoch 5/50, Loss: 275.7594779934988
Epoch 6/50, Loss: 382.30870315709484
Epoch 7/50, Loss: 348.2609767370692
Epoch 8/50, Loss: 319.0104075658594
Epoch 9/50, Loss: 293.0910705602565
Epoch 10/50, Loss: 269.6868924901561
Epoch 11/50, Loss: 248.28805573270873
Epoch 12/50, Loss: 228.54635565001155
Epoch 13/50, Loss: 210.2078050941155
Epoch 14/50, Loss: 193.0782086565357
Epoch 15/50, Loss: 177.00402782009567
Epoch 16/50, Loss: 161.86088582126953
Epoch 17/50, Loss: 147.54617296975047
Epoch 18/50, Loss: 133.97401233015643
Epoch 19/50, Loss: 121.07167642872861
Epoch 20/50, Loss: 108.77694922743088
Epoch 21/50, Loss: 97.0361341609274
Epoch 22/50, Loss: 85.80252071915822
Epoch 23/50, Loss: 75.03518597976708
Epoch 24/50, Loss: 64.69804618260069
Epoch 25/50, Loss: 54.75909809414722
Epoch 26/50, Loss: 45.18980633811926
Epoch 27/50, Loss: 35.96460421802579
Epoch 28/50, Loss: 27.060483621932736
Epoch 

/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


Epoch 40/50, Loss: inf
Epoch 41/50, Loss: inf
Epoch 42/50, Loss: inf
Epoch 43/50, Loss: inf
Epoch 44/50, Loss: inf
Epoch 45/50, Loss: inf
Epoch 46/50, Loss: inf
Epoch 47/50, Loss: inf
Epoch 48/50, Loss: inf
Epoch 49/50, Loss: inf
Epoch 50/50, Loss: inf
Error Rate: 0.10833333333333328
Confusion Matrix:
[[237  63]
 [  2 298]]
lambda =0.04107014028056113
Epoch 1/50, Loss: 1.0818414221373782
Epoch 2/50, Loss: 0.8722140866213515
Epoch 3/50, Loss: 7760.458877014574
Epoch 4/50, Loss: 36245095698.99225
Epoch 5/50, Loss: 4.4038815224044175e+18
Epoch 6/50, Loss: 4.143176847421296e+27
Epoch 7/50, Loss: 1.5910359945778822e+37
Epoch 8/50, Loss: 1.7053344364762308e+47
Epoch 9/50, Loss: 3.9964020006847035e+57
Epoch 10/50, Loss: 1.7340437724084535e+68
Epoch 11/50, Loss: 1.2376680612115735e+79
Epoch 12/50, Loss: 1.3317904455563293e+90
Epoch 13/50, Loss: 2.0222886040054454e+101
Epoch 14/50, Loss: 4.116572929114776e+112
Epoch 15/50, Loss: 1.0786058890356912e+124
Epoch 16/50, Loss: 3.5205269866545554e+135

/usr/local/lib/python3.10/dist-packages/sklearn/utils/_array_api.py:390: RuntimeWarning: invalid value encountered in cast
  return x.astype(dtype, copy=copy, casting=casting)


ValueError: Input y_pred contains NaN.